In [1]:
from langchain_google_genai import GoogleGenerativeAI

c:\Users\Tushar\OneDrive\Desktop\Final Year Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = GoogleGenerativeAI(model = "gemini-pro")  # type: ignore

llm.invoke("What is quantum mechanics?")

''

In [3]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('Data/New_data_txtfiles/', glob="**/*.txt")

In [4]:
docs = loader.load()

In [5]:
len(docs)

8

In [6]:
docs[0]

Document(page_content='Patient ID: 612\n\nAge: 50\n\nBMI: 27\n\nSymptoms: Blurred Vision, Increased Hunger\n\nComplications: Diabetic Retinopathy\n\nHabits: Regular Exercise, Non-Smoker\n\nAllergies: Tree Pollen\n\nMedication: VEGF Inhibitor and GLP-1 Receptor Agonist (Liraglutide)\n\nHow it Treats Symptoms:\n\nVEGF inhibitors address blurred vision. GLP-1 receptor agonists help regulate blood sugar levels, addressing increased hunger.\n\nGuidance on Secondary Complications:\n\nMonitor for signs of diabetic retinopathy. Regular eye check-ups are crucial.\n\nAllergy Consideration:\n\nPatient is allergic to Tree Pollen. Alternative VEGF inhibitors and GLP-1 receptor agonists may be considered. Consult with an ophthalmologist and endocrinologist for suitable alternatives.\n\nRecommendations: Emphasize eye care, lifestyle modifications, and follow prescribed medication regimen.\n\n---\n\nPatient ID: 613\n\nAge: 46\n\nBMI: 27\n\nSymptoms: Tingling Sensation, Increased Thirst\n\nComplication

In [7]:
train_docs = []
for doc in docs:
    if "test" in doc.metadata["source"].lower():
        test_doc = doc
    elif "validation" in  doc.metadata["source"].lower():
        validation_doc = doc
    else:
        train_docs.append(doc)
print(len(train_docs))

6


In [8]:
train_docs[0]

Document(page_content='Patient ID: 1\n\nAge: 40\n\nBMI: 27\n\nSymptoms: Increased Thirst, Fatigue\n\nComplications: None\n\nHabits: Regular Exercise, Non-Smoker\n\nAllergies: None\n\nMedication: DPP-4 Inhibitor (Sitagliptin)\n\n**How it Treats Symptoms:**\n\nDPP-4 inhibitors help regulate blood sugar levels by inhibiting the enzyme DPP-4, which breaks down incretin hormones. This results in increased insulin production and decreased glucagon release, addressing symptoms like increased thirst and fatigue.\n\n**Guidance on Secondary Complications:**\n\nNo secondary complications reported.\n\nRecommendations: Encourage a balanced diet and regular exercise.\n\n---------------------------------------------------------------------\n\nPatient ID: 2\n\nAge: 55\n\nBMI: 31\n\nSymptoms: Frequent Urination, Blurred Vision\n\nComplications: Retinopathy\n\nHabits: Sedentary Lifestyle, Irregular Eating Patterns\n\nAllergies: None\n\nMedication: SGLT2 Inhibitor (Empagliflozin)\n\n**How it Treats Sympt

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function=len,
    is_separator_regex=False,
)

In [10]:
# text_splitter.split_documents(train_docs)

In [11]:
split_training_docs = text_splitter.split_documents(train_docs)

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # type: ignore
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05636945, 0.0048285457, -0.0762591, -0.023642512, 0.05329321]

In [13]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(split_training_docs, embeddings)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [15]:
retriever = db.as_retriever()

template = """You are given a list of past diabetic patient treatments as recommnedations and you are an expert medical practitioner.
Your task is to recommend a treatment for the question asked which will be a patient description including Age, BMI, primary symptoms,
secondary complications and allergies if any present. Generate a custom treatment such that it includes medications for all the symptoms,
complications and allergies if listed along with their description. Use the provided context as a reference. Stick strictly to the context for answering:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
chain.invoke(""" Age: 46
BMI: 27
Symptoms: Joint Pain, Increased Thirst, Dry Eyes, Increased Hunger
Complications: Rheumatoid Arthritis
Habits: Regular Exercise, Non-Smoker
Allergies: Tree Pollen """)

'Medication: DMARD, SGLT2 Inhibitor (Empagliflozin), Artificial Tears, GLP-1 Receptor Agonist (Dulaglutide)\n\nHow it Treats Symptoms:\n- DMARDs address joint pain associated with rheumatoid arthritis.\n- SGLT2 inhibitors help regulate blood sugar levels, addressing increased thirst.\n- Artificial tears address dry eyes.\n- GLP-1 receptor agonists help regulate blood sugar levels, addressing weight loss.\n\nGuidance on Secondary Complications:\n- Monitor for signs of rheumatoid arthritis. Regular check-ups with a rheumatologist are recommended.\n\nAllergy Consideration:\n- Patient is allergic to Tree Pollen. Alternative DMARDs, SGLT2 inhibitors, artificial tears, and GLP-1 receptor agonists may be considered. Consult with a rheumatologist, endocrinologist, and ophthalmologist for suitable alternatives.\n\nRecommendations:\n- Emphasize joint care, lifestyle modifications, and follow prescribed medication regimen.\n- Monitor eye health.'

In [21]:
db.save_local("Gemini_Indexes")